# Python Notebook

In [84]:
import os
import jinja2
import sys
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath("src"))  
import fetchGSheet
from createDataTable import gene_pair0

# Paths
TEMPLATE_PATH = 'HTML/cardTemplate.html'
OUTPUT_DIR = 'data/cards/'

def load_template(template_path):
    """Load Jinja2 template from a file."""
    with open(template_path, 'r') as file:
        return jinja2.Template(file.read())

def prepare_dataframes(gene_pair0):
    """Prepare interaction, ligand, and receptor dataframes."""
    DBlength = len(gene_pair0)
    gene_pair0["Interaction ID"] = [f"CDB{str(i).zfill(4)}" for i in range(1, DBlength + 1)]
    gene_pair0["Interaction Type"] = [
        f'{ligandLocation} {ligand} binds with {receptor} in {receptorLocation}'
        for ligand, ligandLocation, receptor, receptorLocation in zip(
            gene_pair0["Ligand"], gene_pair0["Ligand location"],
            gene_pair0["Receptor"], gene_pair0["Receptor location"]
        )
    ]
    interaction_card = gene_pair0[["Interaction ID", "LR Pair", "Interaction Type", "Perplexity", "PMID support"]]
    interaction_card["Perplexity"] = interaction_card["Perplexity"].str.replace('size=35', 'size=80')

    pop_up_info_lim = fetchGSheet.pop_up_info[
        ["Approved symbol", "Alias symbol", "Previous symbol", "Date symbol changed"]
    ].drop_duplicates(subset="Approved symbol", keep="first")
    
    ligand_card = gene_pair0[["LR Pair", "Ligand", "Ligand name", "Ligand HGNC ID", 'Ligand location']].merge(
        pop_up_info_lim, how='left', left_on='Ligand', right_on='Approved symbol'
    ).drop_duplicates(subset='LR Pair', keep="first").drop(columns=["Ligand"])

    receptor_card = gene_pair0[["LR Pair", "Receptor", "Receptor name", "Receptor HGNC ID", 'Receptor location']].merge(
        pop_up_info_lim, how='left', left_on='Receptor', right_on='Approved symbol'
    ).drop_duplicates(subset='LR Pair', keep="first").drop(columns=["Receptor"])

    return interaction_card, ligand_card, receptor_card

def generate_html_files(template, interaction_card, ligand_card, receptor_card, output_dir):
    """Generate HTML files for each LR Pair."""
    column_values = interaction_card["LR Pair"].dropna().unique()
    os.makedirs(output_dir, exist_ok=True)
    
    for value in column_values:
        value1, value2 = value.split()
        row0 = interaction_card[interaction_card['LR Pair'] == value]
        row1 = ligand_card[ligand_card['LR Pair'] == value]
        row2 = receptor_card[receptor_card['LR Pair'] == value]

        table0_data = row0.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row0.empty else {}
        table1_data = row1.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row1.empty else {}
        table2_data = row2.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row2.empty else {}

        rendered_content = template.render(
            value1=value1,
            value2=value2,
            table0_data=table0_data,
            table1_data=table1_data,
            table2_data=table2_data
        )
        
        output_file = os.path.join(output_dir, f"{value1} {value2}.html")
        with open(output_file, 'w') as file:
            file.write(rendered_content)

# Main execution
if __name__ == "__main__":
    template = load_template(TEMPLATE_PATH)
    interaction_card, ligand_card, receptor_card = prepare_dataframes(gene_pair0)
    generate_html_files(template, interaction_card, ligand_card, receptor_card, OUTPUT_DIR)


/var/folders/r4/pjgb1k_x7fv9xgcb_dg70bnm0000gn/T/ipykernel_6306/356929917.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interaction_card["Perplexity"] = interaction_card["Perplexity"].str.replace('size=35', 'size=80')


In [82]:
import sys
import os
import pandas as pd

# Paths
TEMPLATE_PATH = "HTML/pmidTemplate.html"
OUTPUT_DIR = "data/pubmed/"

# Add the src directory to the path for importing modules
sys.path.append(os.path.abspath("src"))
from createDataTable import gene_pair00

# Load PubMed data
pubmed_data = pd.read_csv("data/pubmed_results.csv")
pubmed_data["Year"] = pubmed_data["Year"].astype(str).str.replace(".0", 
                                                                  "", 
                                                                  regex=False).astype(int)

# Replace spaces in "LR Pair" with a placeholder
gene_pair00["LR Pair"] = gene_pair00["LR Pair"].str.replace(" ", "——")

def load_template(template_path):
    """
    Load the HTML template from a file.

    Parameters:
        template_path (str): Path to the HTML template file.

    Returns:
        str: The contents of the template file as a string.
    """
    with open(template_path, "r") as file:
        return file.read()


def create_detailed_pages_with_tabs(df, gene_column, pmid_column, pubmed_data, template):
    """
    Generate detailed HTML pages with tabs and PubMed links for each gene pair.

    Parameters:
        df (pd.DataFrame): DataFrame containing gene pair information.
        gene_column (str): Name of the column containing gene pair names.
        pmid_column (str): Name of the column containing PMIDs.
        pubmed_data (pd.DataFrame): DataFrame containing PubMed details.
        template (str): HTML template to use for generating the pages.
    """
    for idx, row in df.iterrows():
        gene_name = row[gene_column]
        pmids = row[pmid_column]

        # Ensure PMIDs are properly processed
        sources = [pmid.strip() for pmid in pmids.split(',') if pmid.strip()]

        if sources:
            tab_headers = []
            tab_contents = []

            for i, pmid in enumerate(sources):
                pubmed_row = pubmed_data[pubmed_data["PMID"] == pmid]

                if not pubmed_row.empty:
                    title = pubmed_row["Title"].values[0]
                    abstract = pubmed_row["Abstract"].values[0]
                    journal = pubmed_row["Journal"].values[0]
                    year = pubmed_row["Year"].values[0]
                else:
                    title = "No Title Found"
                    abstract = "No Abstract Found"
                    title = "Journal Unknown"
                    abstract = "Year Unknown"

                active_class = "active" if i == 0 else ""
                tab_headers.append(f'<button class="tablinks {active_class}" onclick="openTab(event, \'tab{pmid}\')">{pmid}</button>')
                tab_contents.append(f"""
                <div id="tab{pmid}" class="tabcontent {active_class}">
                    <h2>{title}</h2>
                    <h3>{journal}; {year}</h3>
                    <p>{abstract}</p>
                    <p><a href="https://pubmed.ncbi.nlm.nih.gov/{pmid}/" target="_blank">[For more details, see PubMed]</a></p>
                </div>
                """)

            # Fill the template with dynamic content
            page_content = template.replace("{{GENE_NAME}}", gene_name)
            page_content = page_content.replace("{{TAB_HEADERS}}", "".join(tab_headers))
            page_content = page_content.replace("{{TAB_CONTENTS}}", "".join(tab_contents))

            # Save the HTML file
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            output_path = os.path.join(OUTPUT_DIR, f"{gene_name}_pmid_details.html")
            with open(output_path, "w") as file:
                file.write(page_content)


if __name__ == "__main__":
    # Load the HTML template
    template = load_template(TEMPLATE_PATH)

    # Generate pages
    create_detailed_pages_with_tabs(
        df=gene_pair00,
        gene_column="LR Pair",
        pmid_column="PMID support",
        pubmed_data=pubmed_data,
        template=template,
    )


In [81]:
gene_pair.columns

Index(['LR Pair', 'Source', 'PMID support', 'Ligand', 'Ligand HGNC ID',
       'Ligand location', 'Receptor', 'Receptor HGNC ID', 'Receptor location',
       'HGNC L R', 'secondary source?', 'PMID link', 'Perplexity',
       'sanity check', 'curator', 'Approved symbol_x', 'Ligand name',
       'Ligand Mouse (MGI) ID', 'Ligand Rat (MGI) ID', 'Approved symbol_y',
       'Approved name', 'Mouse (MGI) ID', 'Rat (RGD) ID'],
      dtype='object', name=0)

In [ ]:
# number of unique vars

lrPairsCount = len(gene_pair["LR Pair"].unique())

ligandCount = len(gene_pair["Ligand"].unique())

receptorCount = len(gene_pair["Receptor"].unique())

gene_pair["PMID support"] = [value.replace(" ", "") for value in gene_pair["PMID support"]]

source = np.array(gene_pair["PMID support"].unique())
source = source.astype(str)
source = ",".join(sorted(set(filter(lambda x: x.lower() != 'nan', source))))

# Split the string into individual elements, filter out empty strings, and get unique values
source = sorted(
    set(filter(lambda x: x.strip() and x.strip().lower() != 'nan', source.split(',')))
)
source = [value.replace(" ", "") for value in source]
sourceCount = len(source)

# for creating PMIDs
gene_pair00 = gene_pair[['LR Pair', 'PMID support']]

# create URLs for the HGNC IDs

# ligand
gene_pair["Ligand HGNC ID"] = [
    '<a href="https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/{}" target="_blank">{}</a>'.format(ligand, ligand)
    for ligand in gene_pair["Ligand HGNC ID"]
]

# receptor
gene_pair["Receptor HGNC ID"] = [
    '<a href="https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/{}" target="_blank">{}</a>'.format(receptor, receptor)
    for receptor in gene_pair["Receptor HGNC ID"]
]

# Perplexity
gene_pair["Perplexity"] = [
    '<a href="{}" target="_blank"> <img src="https://img.icons8.com/?size=35&id=0NbBuNOxUwps&format=png&color=000000" alt="Perplexity AI" /></a>'.format(url)
    for url in gene_pair["Perplexity"]
]

# Function to generate hyperlinks for the "PMID support" column
# Function to generate hyperlinks for the "PMID support" column
def generate_links_with_doi(df, gene_column, pmid_column):
    def create_link(gene, sources):
        # Replace spaces with "——" in the gene name for the link
        gene_name = gene.replace(" ", "——")
        
        if len(sources) == 1:
            source = sources[0]
            if source.startswith("https://www.biorxiv.org/content/"):
                # If the value starts with "https://doi.org/", use it as the hyperlink
                return f'<a href="{source}" target="_blank">BioRxiv preprint</a>'
            else:
                # If it's a single PMID, hyperlink the PMID text
                return f'<a href="https://comp.med.yokohama-cu.ac.jp/collab/connectomeDB/pubmed/{gene_name}_pmid_details.html" target="_blank">{source}</a>'
        else:
            # If multiple PMIDs, show the count and hyperlink to the page
            return f'<a href="https://comp.med.yokohama-cu.ac.jp/collab/connectomeDB/pubmed/{gene_name}_pmid_details.html" target="_blank">{len(sources)} PMIDs</a>'

    # Process each row to generate the "PMID support" column
    df["PMID support"] = [
        create_link(
            gene=row[gene_column], 
            sources=[s.strip() for s in row[pmid_column].split(',') if s.strip()]
        )
        for _, row in df.iterrows()
    ]
    return df


# Generate the links for the "PMID support" column
gene_pair = generate_links_with_doi(gene_pair, gene_column="LR Pair", pmid_column="PMID support")

gene_pair["Mouse (MGI) ID"] = [
        f'<a href="https://www.informatics.jax.org/marker/{mouseOrth}" target="_blank">{mouseOrth}</a>' 
        if pd.notna(mouseOrth) and mouseOrth.strip() else "" 
        for mouseOrth in gene_pair["Mouse (MGI) ID"]
    ]
    
gene_pair["Rat (RGD) ID"] = [
        f'<a href="https://rgd.mcw.edu/rgdweb/report/gene/main.html?id={ratOrth.replace("RGD:", "")}" target="_blank">{ratOrth}</a>' 
        if pd.notna(ratOrth) and ratOrth.strip() else "" 
        for ratOrth in gene_pair["Rat (RGD) ID"]
    ]

In [20]:
# Example usage
create_detailed_pages_with_tabs(df=gene_pair0, gene_column="LR Pair", pmid_column="PMID support", pubmed_data=pubmed_data)

In [ ]:
import sys
import requests
import pandas as pd
import time
import os
sys.path.append(os.path.abspath("src"))  
from createDataTable import source

pmid_list = source
with open("data/ncbi_api_key.txt", "r") as file:
    ncbi_api_key = file.read().strip()  
ncbi_api_key

In [4]:
import sys
import requests
import pandas as pd
import time
import os
sys.path.append(os.path.abspath("src"))  
from createDataTable import source

pmid_list = source
with open("data/ncbi_api_key.txt", "r") as file:
    ncbi_api_key = file.read().strip()  
ncbi_api_key

# Function to fetch PubMed data using PMIDs
def fetch_pubmed_data(pmid_list):
    # Store the results
    results = []
    
    # Base URL for PubMed E-utilities API
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
    
    # Loop through the list of PMIDs and fetch data
    for pmid in pmid_list:
        try:
            # Make request to PubMed API
            params = {
                "db": "pubmed",  # Database is PubMed
                "id": pmid,     # PMID
                "retmode": "xml",  # Return format as XML
                "api_key": ncbi_api_key  # Optional: Get an API key for higher rate limits
            }
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # Check for HTTP errors
            
            # Parse the XML response
            from xml.etree import ElementTree as ET
            tree = ET.ElementTree(ET.fromstring(response.text))
            root = tree.getroot()
            
            # Extract title and abstract
            docsum = root.find("DocSum")
            title = ""
            abstract = "No abstract available"
            
            # Loop through fields in DocSum
            for item in docsum.findall("Item"):
                name = item.get("Name")
                if name == "Title":
                    title = item.text
                elif name == "Source" and not abstract:
                    abstract = "Abstract not available"
            
            # Append the result to the list
            results.append({
                "PMID": pmid,
                "Title": title,
                "Abstract": abstract
            })
        
        except Exception as e:
            print(f"Error fetching data for PMID {pmid}: {e}")
        
        # To avoid hitting the NCBI server too quickly, add a delay between requests
        time.sleep(0.5)

    return results

# Step 3: Create the DataFrame with Results

def create_dataframe(pmid_list):
    # Fetch data
    data = fetch_pubmed_data(pmid_list)

    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Optionally, save to CSV
    df.to_csv("pubmed_results.csv", index=False)
    
    return df


# Fetch and create the table
df = create_dataframe(pmid_list)

# Display the results
print(df)

Error fetching data for PMID 97517301: 'NoneType' object has no attribute 'findall'
Error fetching data for PMID https://doi.org/10.1101/2022.07.27.501694: 'NoneType' object has no attribute 'findall'
          PMID                                              Title  \
0     10025398  The integrin alpha v beta 6 binds and activate...   
1     10037686  Identification of a novel activation-inducible...   
2     10037743  Heparan sulfate-modified CD44 promotes hepatoc...   
3     10050855  Vertebrate Hedgehog signalling modulated by in...   
4     10051567  Angiopoietins 3 and 4: diverging gene counterp...   
...        ...                                                ...   
2273   9972281  Insulin-like growth factor-I receptor signal t...   
2274   9973222  Activation of neurotrophin-3 receptor TrkC ind...   
2275   9988678  Decorin is a biological ligand for the epiderm...   
2276   9988761  Identification of residues within the 727-767 ...   
2277   9990065  Prototypic G protein-cou

In [ ]:
import sys
import requests
import pandas as pd
import time
import os
sys.path.append(os.path.abspath("src"))  
from createDataTable import source

pmid_list=source

# Read the API key from a file
with open("data/ncbi_api_key.txt", "r") as file:
    ncbi_api_key = file.read().strip()

# Function to fetch PubMed data using PMIDs
def fetch_pubmed_data(pmid_list):
    results = []
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    
    for pmid in pmid_list:
        try:
            # Request parameters for efetch API (note: db=pubmed, rettype=abstract)
            params = {
                "db": "pubmed",
                "id": pmid,         # PMID
                "retmode": "xml",    # Return as XML
                "api_key": ncbi_api_key,  # Use the API key
                "rettype": "abstract"  # Request the abstract
            }
            
            # Fetch data from PubMed using efetch
            response = requests.get(base_url, params=params)
            response.raise_for_status()  # Raise an exception for bad responses
            
            # Parse the XML response
            tree = ET.ElementTree(ET.fromstring(response.text))
            root = tree.getroot()
            
            # Extract information from the XML response
            docsum = root.find("PubmedArticle")
            title = ""
            abstract = "No abstract available"
            
            # Extract title
            title_tag = docsum.find(".//ArticleTitle")
            if title_tag is not None:
                title = title_tag.text
            
            # Extract abstract (if available)
            abstract_tag = docsum.find(".//AbstractText")
            if abstract_tag is not None:
                abstract = abstract_tag.text
            
            # Append to results list
            results.append({
                "PMID": pmid,
                "Title": title,
                "Abstract": abstract
            })
        
        except Exception as e:
            print(f"Error fetching data for PMID {pmid}: {e}")
        
        # Rate limit requests to avoid overloading the server
        time.sleep(0.5)

    return results
    
# Fetch data and create DataFrame
def create_dataframe(pmid_list):
    data = fetch_pubmed_data(pmid_list)
    df = pd.DataFrame(data)
    df.to_csv("data/pubmed_results.csv", index=False)  # Optional CSV export
    return df

# Display the results
df = create_dataframe(pmid_list)
print(df)


Error fetching data for PMID 10189055: not well-formed (invalid token): line 7, column 71
Error fetching data for PMID 10196546: not well-formed (invalid token): line 7, column 71
Error fetching data for PMID 11116144: not well-formed (invalid token): line 7, column 71
Error fetching data for PMID 12794140: not well-formed (invalid token): line 7, column 71
Error fetching data for PMID 12796776: not well-formed (invalid token): line 7, column 71
Error fetching data for PMID 97517301: 'NoneType' object has no attribute 'find'


In [ ]:
# import os
import jinja2
import sys
import pandas as pd
sys.path.append(os.path.abspath("src"))  
import fetchGSheet
from createDataTable import gene_pair0
import numpy as np


DBlength = len(gene_pair0)
# Generate Interaction IDs
gene_pair0["Interaction ID"] = [f"CDB{str(i).zfill(4)}" for i in range(1, DBlength + 1)]

interaction_card = gene_pair0[["Interaction ID", "LR Pair","Perplexity", "PMID support"]]
interaction_card.loc[:, 'Perplexity'] = interaction_card['Perplexity'].str.replace('size=35', 'size=100')

ligand_card = gene_pair0[["LR Pair", "Ligand", "Ligand name", "Ligand HGNC ID", 'Ligand location']]
column_values = gene_pair0["LR Pair"].tolist()
column_values = [value for value in column_values if value not in [None, "", np.nan]]


pop_up_info_lim = fetchGSheet.pop_up_info[["Approved symbol", "Alias symbol",
                                           "Mouse genome informatics (MGI) ID",
                                           "Rat genome database (RGD) ID",
                                           "Previous symbol",
                                           "Date symbol changed"]]
pop_up_info_lim = pop_up_info_lim.drop_duplicates(subset="Approved symbol", 
                                                  keep="first")
pop_up_info_lim["Mouse genome informatics (MGI) ID"] = [
    '<a href="https://www.informatics.jax.org/marker/{}" target="_blank">{}</a>'.format(mouseOrth, mouseOrth)
    if not (pd.isna(mouseOrth) or mouseOrth == " ")
    else ""
    for mouseOrth in pop_up_info_lim["Mouse genome informatics (MGI) ID"]
]

pop_up_info_lim["Rat genome database (RGD) ID"] = [
    '<a href="https://rgd.mcw.edu/rgdweb/report/gene/main.html?id={}" target="_blank">{}</a>'.format(ratOrth.replace("RGD:", ""), ratOrth)
    if not (pd.isna(ratOrth) or ratOrth == " ")
    else ""
    for ratOrth in pop_up_info_lim["Rat genome database (RGD) ID"]
]

ligand_card = ligand_card.merge(pop_up_info_lim, how='left', left_on='Ligand', right_on='Approved symbol')
ligand_card = ligand_card.drop_duplicates(subset='LR Pair', 
                                                  keep="first")

ligand_card = ligand_card.drop(columns=["Ligand"])


receptor_card = gene_pair0[["LR Pair", "Receptor", "Receptor name", "Receptor HGNC ID", 'Receptor location']]
receptor_card = receptor_card.merge(pop_up_info_lim, how='left', left_on='Receptor', right_on='Approved symbol')
receptor_card = receptor_card.drop_duplicates(subset='LR Pair', 
                                                  keep="first")
receptor_card = receptor_card.drop(columns=["Receptor"])

# Define Jinja2 template for cards
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        .title {
            text-align: center; /* Centers the title horizontally */
            font-size: 30px;
            font-weight: bold;
            margin-bottom: 20px; /* Adds some space below the title */
        }
        body {
            font-family: Arial, sans-serif;
            margin: 24px;
            background-color: #f4f4f4;
        }
        .card-container {
            display: flex;
            justify-content: space-between;
            flex-wrap: wrap;
            gap: 15px;
            margin: 0 auto;
            max-width: 1200px;
        }
        .card {
            width: 48%;
            height: auto;
            padding: 20px;
            border: 1px solid #ccc;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            background-color: white;
            box-sizing: border-box;
        }
        .card-full-row {
            width: 100%;
            height: auto;
            padding: 20px;
            border: 1px solid #ccc;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            background-color: white;
            box-sizing: border-box;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .card h2 {
            text-align: center;
            size: 30px;
            margin-bottom: 30px;
        }

        .card-full-row h2 {
            text-align: left;
            size: 30px
            margin-bottom: 30px;
        }
        .card p {
            text-align: center;
            color: #666;
        }
        .card-full-row .column {
            margin: 10px 0;
            font-size: 24px;
        }
        .card .column {
            margin: 10px 0;
            font-size: 20px;
        }
        .card .column strong {
            font-weight: bold;
        }
        .card-content {
            flex: 1; /* Left side */
        }
        .card-right {
            flex: 1; /* Right side */
            display: flex;
            flex-direction: column;
            align-items: flex-end; /* Align the content to the right */
            justify-content: center;
            text-align: center;
        }
    </style>
</head>
<body>
    <h1 style="text-align: center; font-size: 30px; font-weight: bold; width: 100%;">{{ value1 }} —— {{ value2 }} </h1>
    <div class="card-container">
        <div class="card-full-row">
            <div class="card-content">
                <h2>{{ value1 }} —— {{ value2 }} Interaction</h2>
                {% for column, value in table0_data.items() %}
                    {% if column != "Perplexity" %}
                        <div class="column">
                            <strong>{{ column }}:</strong> {{ value }}
                        </div>
                    {% endif %}
                {% endfor %}
            </div>
            <div class="card-right">
                {% if "Perplexity" in table0_data %}
                    <div>
                        {{ table0_data["Perplexity"] }}
                    </div>
                {% endif %}
            </div>
        </div>
    </div>
    <br>
    <div class="card-container">
        <div class="card">
            <h2>Ligand: {{ value1 }}</h2>
            {% for column, value in table1_data.items() %}
                <div class="column">
                    <strong>{{ column }}:</strong> 
                    {{ value }}
                </div>
            {% endfor %}
        </div>
        <div class="card">
            <h2>Receptor: {{ value2 }}</h2>
            {% for column, value in table2_data.items() %}
                <div class="column">
                    <strong>{{ column }}:</strong> 
                    {{ value }}
                </div>
            {% endfor %}
        </div>
    </div>
</body>
</html>

"""

# Define output directory where HTML files will be saved
output_dir = '_site/cards/'  # Replace with actual output path

# Loop through each LR pair in the data tables
for value in column_values:  # column_values should be the list of "LR Pair" values, e.g., from one of your tables
    # Split the value into two parts (e.g., "SAA3P LY96" -> ["SAA3P", "LY96"])
    value1, value2 = value.split()

    # Find the matching rows from both tables (assuming data tables are pandas DataFrames)
    row0 = interaction_card[interaction_card['LR Pair'] == value]
    row1 = ligand_card[ligand_card['LR Pair'] == value]
    row2 = receptor_card[receptor_card['LR Pair'] == value]

    # Extract all relevant data from both tables for this LR pair
    table0_data = row0.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row0.empty else {}
    table1_data = row1.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row1.empty else {}
    table2_data = row2.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row2.empty else {}

    # Render the template with the extracted values from both tables
    rendered_content = jinja2.Template(template).render(
        value1=value1,
        value2=value2,
        table0_data=table0_data,
        table1_data=table1_data,
        table2_data=table2_data
    )
    
    # Save the rendered content as an HTML file
    output_file = os.path.join(output_dir, f"{value1} {value2}.html")
    with open(output_file, 'w') as f:
        f.write(rendered_content)

In [4]:
pip install biopython

  Using cached biopython-1.84.tar.gz (25.8 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [899 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/MaxEntropy.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/LogisticRegression.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/SeqFeature.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/SeqRecord.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/MarkovModel.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/pairwise2.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/__init__.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      copying Bio/File.py -> build/lib.macosx-11.0-arm64-cpython-313/Bio
      cop

In [ ]:
import os
os.getcwd()

In [ ]:
import os
import jinja2
import sys
sys.path.append(os.path.abspath("src"))  
import fetchGSheet
from createDataTable import gene_pair0
import numpy as np
gene_pair0.columns

In [ ]:
gene_pair0

In [ ]:
# Define Jinja2 template for cards
template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ value1 }} & {{ value2 }} - Card Page</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .card-container { display: flex; flex-wrap: wrap; }
        .card { width: 200px; height: auto; margin: 10px; padding: 10px; border: 1px solid #ccc; border-radius: 8px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); }
        .card h3 { text-align: center; }
        .card p { text-align: center; color: #666; }
        .card .column { margin: 5px 0; }
    </style>
</head>
<body>
    <h1>Cards for {{ value1 }} and {{ value2 }}</h1>
    <div class="card-container">
        <div class="card">
            <h3>{{ value1 }}</h3>
            <p>Description or info about {{ value1 }}.</p>
            {% for column, value in table1_data.items() %}
                <div class="column"><strong>{{ column }}:</strong> {{ value }}</div>
            {% endfor %}
        </div>
        <div class="card">
            <h3>{{ value2 }}</h3>
            <p>Description or info about {{ value2 }}.</p>
            {% for column, value in table2_data.items() %}
                <div class="column"><strong>{{ column }}:</strong> {{ value }}</div>
            {% endfor %}
        </div>
    </div>
</body>
</html>
"""

# Define output directory where HTML files will be saved
output_dir = 'data/cards/'  # Replace with actual output path

# Loop through each LR pair in the data tables
for value in column_values:  # column_values should be the list of "LR Pair" values, e.g., from one of your tables
    # Split the value into two parts (e.g., "SAA3P LY96" -> ["SAA3P", "LY96"])
    value1, value2 = value.split()

    # Find the matching rows from both tables (assuming data tables are pandas DataFrames)
    row1 = ligand_card[ligand_card['LR Pair'] == value]
    row2 = receptor_card[receptor_card['LR Pair'] == value]

    # Extract all relevant data from both tables for this LR pair
    table1_data = row1.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row1.empty else {}
    table2_data = row2.drop('LR Pair', axis=1).to_dict(orient='records')[0] if not row2.empty else {}

    # Render the template with the extracted values from both tables
    rendered_content = jinja2.Template(template).render(
        value1=value1,
        value2=value2,
        table1_data=table1_data,
        table2_data=table2_data
    )
    
    # Save the rendered content as an HTML file
    output_file = os.path.join(output_dir, f"{value1} {value2}.html")
    with open(output_file, 'w') as f:
        f.write(rendered_content)

In [ ]:
from itables import init_notebook_mode
import pandas as pd
from itables import show
from itables import options
from IPython.display import HTML, display
import numpy as np
import gspread
from google.oauth2.service_account import Credentials

from itables import init_notebook_mode, show
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

sheet_ID = "15FfI7cVpJmAcytTBmhVE2Z77tgVQYfEZUEe700_Wzlg"
credentials_file = 'data/connectomedb2025-a9acdf562a84.json'

# Function to fetch data from a Google Sheet tab
def fetch_google_sheet_data(sheet_ID, tab_name, credentials_file):
    """
    Fetch data from a specific tab of a Google Sheet and return it as a pandas DataFrame.
    
    Parameters:
        sheet_ID (str): The ID of the Google Sheet.
        tab_name (str): The name of the tab/worksheet to fetch.
        credentials_file (str): Path to the service account JSON credentials file.
    
    Returns:
        pd.DataFrame: Data from the specified Google Sheet tab.
    """
    # Define the scopes and authenticate
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(credentials_file, scopes=SCOPES)
    client = gspread.authorize(credentials)

    # Open the Google Sheet and get the specified tab
    try:
        sheet = client.open_by_key(sheet_ID)
        worksheet = sheet.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        raise ValueError(f"Tab '{tab_name}' not found in Google Sheet with ID '{sheet_ID}'.")

    # Get all values from the tab and convert to DataFrame
    data = worksheet.get_all_values()
    df = pd.DataFrame(data)
    df = pd.DataFrame(data[1:], columns=data[0])  
    return df

# Fetching data from Google Sheets
gene_pair = fetch_google_sheet_data(sheet_ID, "Literature supported", credentials_file)
pop_up_info = fetch_google_sheet_data(sheet_ID, "HGNC_Dec2024", credentials_file)
gene_pair


In [ ]:
len(gene_pair[1:])

In [ ]:
# Select only the relevant columns from pop_up_info
pop_up_info_lim = pop_up_info[["Approved symbol", "Approved name"]]
pop_up_info_lim = pop_up_info_lim.drop_duplicates(subset="Approved symbol", keep="first")

# Drop columns where all values are NA in gene_pair
gene_pair = gene_pair.dropna(axis=1, how='all')

# Rename columns for better clarity
gene_pair = gene_pair.rename(columns={
    "Ligand receptor pair": "LR Pair",
    "Ligand gene symbol": "Ligand(L)",
    "Receptor gene symbol": "Receptor(R)",
    "Perplexity link": "Perplexity"
})

# Merge gene_pair with pop_up_info_lim for Ligand(L)
gene_pair = gene_pair.merge(pop_up_info_lim, how='left', left_on='Ligand(L)', right_on='Approved symbol')

gene_pair = gene_pair.rename(columns={"Approved name": "Ligand name"})

gene_pair = gene_pair.merge(pop_up_info_lim, how='left', left_on='Receptor(R)', right_on='Approved symbol')

gene_pair = gene_pair.rename(columns={"Approved name": "Receptor name"})

gene_pair = gene_pair.drop(columns=["Approved symbol_x", "Approved symbol_y"])

# Drop columns where all values are NA in gene_pair
#gene_pair = gene_pair.dropna(axis=1, how='all')

gene_pair = gene_pair.fillna(" ")
gene_pair = gene_pair[gene_pair['LR Pair'] != ' ']
gene_pair

In [ ]:
gene_pair.columns

In [ ]:
sourceCount =np.array(gene_pair["PMID support"].unique())
sourceCount = sourceCount.astype(str)
sourceCount = ",".join(sorted(set(filter(lambda x: x.lower() != 'nan', sourceCount))))

# Split the string into individual elements, filter out empty strings, and get unique values
sourceCount = sorted(
    set(filter(lambda x: x.strip() and x.strip().lower() != 'nan', sourceCount.split(',')))
)
sourceCount = [value.replace(" ", "") for value in sourceCount]

In [ ]:
len(sourceCount)

In [ ]:
gene_pair = gene_pair[first_columns + [col for col in gene_pair.columns if col not in first_columns]]
gene_pair

In [ ]:
gene_pair = gene_pair[first_columns, rem_columns]

In [ ]:
gene_pair.rename(columns={'Ligand Location': '<span title="This is the tooltip text">Ligand Location</span>'}, inplace=True)

In [ ]:
gene_pair.columns

In [ ]:
def add_row(change):
    global gene_pair
    # Add a new row at the top with None values
    new_row = {col: None for col in gene_pair.columns}
    gene_pair = pd.DataFrame([new_row] + gene_pair.to_dict(orient="records"))
    update_table()

# Function to remove the last row of the dataframe
def remove_row(change):
    global gene_pair
    if len(gene_pair) > 0:
        gene_pair = gene_pair[:-1]  # Remove the last row
        update_table()

In [ ]:
from itables import init_notebook_mode
import pandas as pd
from itables import show
from itables import options
from IPython.display import HTML, display
import numpy as np
import gspread
from google.oauth2.service_account import Credentials

# Define the scope
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# Load the service account credentials
credentials = Credentials.from_service_account_file(
    'data/connectomedb2025-a9acdf562a84.json',
    scopes=SCOPES
)

# Set sheet ID
sheet_ID = "https://docs.google.com/spreadsheets/d/1QZuIgph6JcxwSXgWGCnwAHCxqdxtGD2jNULsfPMYv_o/edit?gid=0#gid=0"

tab_name = "Literature supported"

# Authenticate with gspread
client = gspread.authorize(credentials)

# Open the Google Sheet by URL
sheet = client.open_by_url(sheet_ID)

# Access a specific tab (worksheet) by name
gene_pair = sheet.worksheet(tab_name)

In [ ]:
from itables import init_notebook_mode
import pandas as pd
from itables import show
from itables import options
from IPython.display import HTML, display
import numpy as np
import gspread
from google.oauth2.service_account import Credentials

from itables import init_notebook_mode, show
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

sheet_ID = "15FfI7cVpJmAcytTBmhVE2Z77tgVQYfEZUEe700_Wzlg"
credentials_file = 'data/connectomedb2025-a9acdf562a84.json'

# Function to fetch data from a Google Sheet tab
def fetch_google_sheet_data(sheet_ID, tab_name, credentials_file):
    """
    Fetch data from a specific tab of a Google Sheet and return it as a pandas DataFrame.
    
    Parameters:
        sheet_ID (str): The ID of the Google Sheet.
        tab_name (str): The name of the tab/worksheet to fetch.
        credentials_file (str): Path to the service account JSON credentials file.
    
    Returns:
        pd.DataFrame: Data from the specified Google Sheet tab.
    """
    # Define the scopes and authenticate
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(credentials_file, scopes=SCOPES)
    client = gspread.authorize(credentials)

    # Open the Google Sheet and get the specified tab
    try:
        sheet = client.open_by_key(sheet_ID)
        worksheet = sheet.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        raise ValueError(f"Tab '{tab_name}' not found in Google Sheet with ID '{sheet_ID}'.")

    # Get all values from the tab and convert to DataFrame
    data = worksheet.get_all_values()
    df = pd.DataFrame(data)
    df.columns = df.iloc[0]  # Set the first row as the header
    df = df[1:].reset_index(drop=True)  # Remove the header row from the data
    return df

gene_pair = fetch_google_sheet_data(sheet_ID, "Literature supported", credentials_file)
pop_up_info = fetch_google_sheet_data(sheet_ID, "HGNC_Dec2024", credentials_file)

# Drop columns where all values are NA
gene_pair = gene_pair.dropna(axis=1, how='all')
gene_pair = gene_pair.fillna(" ")

# Rename columns
gene_pair = gene_pair.rename(columns={
    "Ligand receptor pair": "Ligand/Receptor",
    "Ligand gene symbol": "Ligand",
    "Receptor gene symbol": "Receptor",
})

gene_pair

In [ ]:
gene_pair

In [ ]:
tab

In [ ]:
from itables import init_notebook_mode
import pandas as pd
from itables import show
from itables import options
from IPython.display import HTML, display
import numpy as np
import gspread
from google.oauth2.service_account import Credentials

from itables import init_notebook_mode, show
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

sheet_ID = "15FfI7cVpJmAcytTBmhVE2Z77tgVQYfEZUEe700_Wzlg"
credentials_file = 'data/connectomedb2025-a9acdf562a84.json'

# Function to fetch data from a Google Sheet tab
def fetch_google_sheet_data(sheet_ID, tab_name, credentials_file):
    """
    Fetch data from a specific tab of a Google Sheet and return it as a pandas DataFrame.
    
    Parameters:
        sheet_ID (str): The ID of the Google Sheet.
        tab_name (str): The name of the tab/worksheet to fetch.
        credentials_file (str): Path to the service account JSON credentials file.
    
    Returns:
        pd.DataFrame: Data from the specified Google Sheet tab.
    """
    # Define the scopes and authenticate
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(credentials_file, scopes=SCOPES)
    client = gspread.authorize(credentials)

    # Open the Google Sheet and get the specified tab
    try:
        sheet = client.open_by_key(sheet_ID)
        worksheet = sheet.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        raise ValueError(f"Tab '{tab_name}' not found in Google Sheet with ID '{sheet_ID}'.")

    # Get all values from the tab and convert to DataFrame
    data = worksheet.get_all_values()
    df = pd.DataFrame(data)
    df.columns = df.iloc[0]  # Set the first row as the header
    df = df[1:].reset_index(drop=True)  # Remove the header row from the data
    return df

In [ ]:
from itables import init_notebook_mode
import pandas as pd
from itables import show
from itables import options
from IPython.display import HTML, display
import numpy as np
import gspread
from google.oauth2.service_account import Credentials

from itables import init_notebook_mode, show
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

sheet_ID = "15FfI7cVpJmAcytTBmhVE2Z77tgVQYfEZUEe700_Wzlg"
credentials_file = 'data/connectomedb2025-a9acdf562a84.json'

# Function to fetch data from a Google Sheet tab
def fetch_google_sheet_data(sheet_ID, tab_name, credentials_file):
    """
    Fetch data from a specific tab of a Google Sheet and return it as a pandas DataFrame.
    
    Parameters:
        sheet_ID (str): The ID of the Google Sheet.
        tab_name (str): The name of the tab/worksheet to fetch.
        credentials_file (str): Path to the service account JSON credentials file.
    
    Returns:
        pd.DataFrame: Data from the specified Google Sheet tab.
    """
    # Define the scopes and authenticate
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    credentials = Credentials.from_service_account_file(credentials_file, scopes=SCOPES)
    client = gspread.authorize(credentials)

    # Open the Google Sheet and get the specified tab
    try:
        sheet = client.open_by_key(sheet_ID)
        worksheet = sheet.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        raise ValueError(f"Tab '{tab_name}' not found in Google Sheet with ID '{sheet_ID}'.")

    # Get all values from the tab and convert to DataFrame
    data = worksheet.get_all_values()
    df = pd.DataFrame(data)
    df.columns = df.iloc[0]  # Set the first row as the header
    df = df[1:].reset_index(drop=True)  # Remove the header row from the data
    return df

# Fetching data from Google Sheets
gene_pair = fetch_google_sheet_data(sheet_ID, "Literature supported", credentials_file)
pop_up_info = fetch_google_sheet_data(sheet_ID, "HGNC_Dec2024", credentials_file)

# Select only the relevant columns from pop_up_info
pop_up_info_lim = pop_up_info[["Approved symbol", "Approved name"]]
pop_up_info_lim = pop_up_info_lim.drop_duplicates(subset="Approved symbol", keep="first")

# Drop columns where all values are NA in gene_pair
gene_pair = gene_pair.dropna(axis=1, how='all')
gene_pair = gene_pair.fillna(" ")

# Rename columns for better clarity
gene_pair = gene_pair.rename(columns={
    "Ligand receptor pair": "LR Pair",
    "Ligand gene symbol": "Ligand(L)",
    "Receptor gene symbol": "Receptor(R)",
})

# Merge gene_pair with pop_up_info_lim for Ligand(L)
gene_pair = gene_pair.merge(pop_up_info_lim, how='left', left_on='Ligand(L)', right_on='Approved symbol')

gene_pair = gene_pair.rename(columns={"Approved name": "Ligand name"})

gene_pair = gene_pair.merge(pop_up_info_lim, how='left', left_on='Receptor(R)', right_on='Approved symbol')

gene_pair = gene_pair.rename(columns={"Approved name": "Receptor name"})

gene_pair = gene_pair.drop(columns=["Approved symbol_x", "Approved symbol_y"])

# Drop rows where all values are NA in all columns
gene_pair = gene_pair.dropna(how='all')

if "PMID link" in gene_pair.columns:
    gene_pair = gene_pair.drop(columns=["PMID link"])

# Add
first_columns=['LR Pair','Source', 'Ligand(L)','Receptor(R)']
end_columns=['secondary source?']
gene_pair = gene_pair[first_columns + [col for col in gene_pair.columns if col not in first_columns + end_columns] + end_columns]

# number of unique vars

lrPairsCount = len(gene_pair["LR Pair"].unique())

ligandCount = len(gene_pair["Ligand(L)"].unique())

receptorCount = len(gene_pair["Receptor(R)"].unique())

gene_pair["PMID support"] = [value.replace(" ", "") for value in gene_pair["PMID support"]]

sourceCount = np.array(gene_pair["PMID support"].unique())
sourceCount = sourceCount.astype(str)
sourceCount = ",".join(sorted(set(filter(lambda x: x.lower() != 'nan', sourceCount))))

# Split the string into individual elements, filter out empty strings, and get unique values
sourceCount = sorted(
    set(filter(lambda x: x.strip() and x.strip().lower() != 'nan', sourceCount.split(',')))
)
sourceCount = [value.replace(" ", "") for value in sourceCount]
sourceCount = len(sourceCount)

# create URLs for the HGNC IDs


# PMID support
gene_pair["PMID support"] = [
    ', '.join(
        f'<a href="https://pubmed.ncbi.nlm.nih.gov/{sources.strip()}/" target="_blank">{sources.strip()}</a>'
        for sources in source.split(',') if sources.strip()
    )
    for source in gene_pair["PMID support"]
]


# ligand
gene_pair["Ligand HGNC ID"] = [
    '<a href="https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/{}" target="_blank">{}</a>'.format(ligand, ligand)
    for ligand in gene_pair["Ligand HGNC ID"]
]

# gene symbol
gene_pair["Ligand(L)"] = [
    f'<span title="{ligand_name}">{ligand_symbol}</span>'
    for ligand_name, ligand_symbol in zip(gene_pair["Ligand name"], gene_pair["Ligand(L)"])
]

# receptor
gene_pair["Receptor HGNC ID"] = [
    '<a href="https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/{}" target="_blank">{}</a>'.format(receptor, receptor)
    for receptor in gene_pair["Receptor HGNC ID"]
]

# gene symbol
gene_pair["Receptor(R)"] = [
    f'<span title="{receptor_name}">{receptor_symbol}</span>'
    for receptor_name, receptor_symbol in zip(gene_pair["Receptor name"], gene_pair["Receptor(R)"])
]
gene_pair["Ligand location"] = [
    '<span title="This is the tooltip for {}">{}</span>'.format(loc, loc)
    for loc in gene_pair["Ligand location"]
]

gene_pair["Receptor location"] = [
    '<span title="This is the tooltip for {}">{}</span>'.format(loc, loc)
    for loc in gene_pair["Receptor location"]
]

In [ ]:
gene_pair.columns

In [ ]:
gene_pair["Perplexity link"]